<a href="https://colab.research.google.com/github/rupaknandy1996-dotcom/csir_project/blob/main/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import zipfile
import os

zip_path = "/content/ERA5_All_Temperature_Trends.zip"
extract_path = "/content/ERA5_All_Temperature_Trends"

# Unzip
with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(extract_path)

print("✅ Unzipped to:", extract_path)

# Show contents
for root, dirs, files in os.walk(extract_path):
    level = root.replace(extract_path, "").count(os.sep)
    indent = " " * 4 * level
    print(f"{indent}{os.path.basename(root)}/")
    subindent = " " * 4 * (level + 1)
    for f in files:
        print(f"{subindent}{f}")


FileNotFoundError: [Errno 2] No such file or directory: '/content/ERA5_All_Temperature_Trends.zip'

In [ ]:
# ============================================
# CLIP ERA5 TEMP FILES WITH GANGA BASIN SHP
# ORGANIZE INTO 4 FOLDERS + RENAME
# ============================================

import os
import glob
import geopandas as gpd
import rasterio
from rasterio.mask import mask
from shapely.geometry import mapping


# ===============================
# PATHS
# ===============================

shp_path = "/content/1a_GRB_WGS84.shp"

base_dir = "/content/ERA5_All_Temperature_Trends/ERA5_All_Temperature_Trends"

t2m_dir  = os.path.join(base_dir, "T2M_Trend")
t300_dir = os.path.join(base_dir, "T300_MONTHLY_TRENDS_ZYP")
t500_dir = os.path.join(base_dir, "T500_MONTHLY_TRENDS_ZYP_FINAL")
t850_dir = os.path.join(base_dir, "T850_MONTHLY_TRENDS_ZYP_FINAL")


# ===============================
# OUTPUT FOLDERS
# ===============================

out_base = "/content/CLIPPED_TEMP"

out_t2m  = os.path.join(out_base, "CLIPPED_T2M")
out_t300 = os.path.join(out_base, "CLIPPED_T300")
out_t500 = os.path.join(out_base, "CLIPPED_T500")
out_t850 = os.path.join(out_base, "CLIPPED_T850")

os.makedirs(out_t2m, exist_ok=True)
os.makedirs(out_t300, exist_ok=True)
os.makedirs(out_t500, exist_ok=True)
os.makedirs(out_t850, exist_ok=True)


# ===============================
# LOAD SHAPEFILE
# ===============================

gdf = gpd.read_file(shp_path)
shapes = [mapping(geom) for geom in gdf.geometry]


# ===============================
# MONTH DICTIONARY
# ===============================

month_map = {
    "JAN":"01","FEB":"02","MAR":"03","APR":"04",
    "MAY":"05","JUN":"06","JUL":"07","AUG":"08",
    "SEP":"09","OCT":"10","NOV":"11","DEC":"12"
}


# ===============================
# CLIP FUNCTION
# ===============================

def clip_and_save(infile, outfolder, prefix):

    with rasterio.open(infile) as src:

        clipped, transform = mask(
            src, shapes,
            crop=True,
            nodata=src.nodata
        )

        meta = src.meta.copy()

        meta.update({
            "height": clipped.shape[1],
            "width": clipped.shape[2],
            "transform": transform,
            "nodata": src.nodata
        })


        fname = os.path.basename(infile)


        # ---------- EXTRACT MONTH ----------
        month = None
        for m in month_map.keys():
            if m in fname.upper():
                month = m
                break


        # ---------- EXTRACT TYPE ----------
        if "SEN" in fname.upper():
            var = "SEN"
        elif "PVAL" in fname.upper():
            var = "PVAL"
        elif "TFPW" in fname.upper():
            var = "TFPW"
        else:
            var = "DATA"


        # ---------- NEW NAME ----------
        if month is not None:
            new_name = f"{prefix}_{month}_{var}.tif"
        else:
            new_name = f"{prefix}_{var}.tif"


        out_path = os.path.join(outfolder, new_name)


        # ---------- SAVE ----------
        with rasterio.open(out_path, "w", **meta) as dst:
            dst.write(clipped)


        print("✅ Saved:", new_name)



# ===============================
# PROCESS FOLDERS
# ===============================

print("🚀 Processing T2M...")

for f in glob.glob(t2m_dir + "/*.tif"):
    clip_and_save(f, out_t2m, "T2M")


print("🚀 Processing T300...")

for f in glob.glob(t300_dir + "/*.tif"):
    clip_and_save(f, out_t300, "T300")


print("🚀 Processing T500...")

for f in glob.glob(t500_dir + "/*.tif"):
    clip_and_save(f, out_t500, "T500")


print("🚀 Processing T850...")

for f in glob.glob(t850_dir + "/*.tif"):
    clip_and_save(f, out_t850, "T850")


print("\n🎉 ALL FILES CLIPPED & ORGANIZED!")
print("📂 Output Folder:", out_base)


In [ ]:
import shutil
from google.colab import files

# Folder to zip
folder = "/content/CLIPPED_TEMP"
zip_path = "/content/CLIPPED_TEMP_FINAL.zip"

# Create zip
shutil.make_archive(zip_path.replace(".zip",""), "zip", folder)

print("✅ ZIP created:", zip_path)

# Download
files.download(zip_path)


In [ ]:
import os

folder = "/content/ERA5_All_Temperature_Trends/ERA5_All_Temperature_Trends/T2M_Trend"

files = sorted(os.listdir(folder))

print("📂 Total files:", len(files), "\n")

for f in files:
    print(os.path.join(folder, f))


In [ ]:
# ============================================
# SAFE CLIP + RENAME T2M (HANDLES DEC FORMAT)
# ============================================

import os
import glob
import re
import geopandas as gpd
import rasterio
from rasterio.mask import mask
from shapely.geometry import mapping


# ===============================
# PATHS
# ===============================

shp_path = "/content/1a_GRB_WGS84.shp"

input_dir = "/content/ERA5_All_Temperature_Trends/ERA5_All_Temperature_Trends/T2M_Trend"

out_dir = "/content/CLIPPED_T2M_RENAMED_V2"

os.makedirs(out_dir, exist_ok=True)


# ===============================
# LOAD SHAPEFILE
# ===============================

gdf = gpd.read_file(shp_path)
shapes = [mapping(geom) for geom in gdf.geometry]


# ===============================
# MONTH MAP
# ===============================

month_map_num = {
    "01":"JAN","02":"FEB","03":"MAR","04":"APR",
    "05":"MAY","06":"JUN","07":"JUL","08":"AUG",
    "09":"SEP","10":"OCT","11":"NOV"
}

month_map_txt = {
    "jan":"JAN","feb":"FEB","mar":"MAR","apr":"APR",
    "may":"MAY","jun":"JUN","jul":"JUL","aug":"AUG",
    "sep":"SEP","oct":"OCT","nov":"NOV","dec":"DEC"
}


# ===============================
# CLIP FUNCTION
# ===============================

def get_month(fname):

    fname = fname.lower()


    # Case 1: month_01 ... month_11
    m1 = re.search(r"month_(\d{2})", fname)

    if m1:
        return month_map_num.get(m1.group(1))


    # Case 2: t2m_Dec
    for k in month_map_txt:
        if f"_{k}_" in fname:
            return month_map_txt[k]


    return "UNK"


def get_var(fname):

    fname = fname.lower()

    if "pvalue" in fname:
        return "PVAL"

    if "senslope" in fname:
        return "SEN"

    if "tfpw" in fname:
        return "TFPW"

    return "DATA"



def clip_rename_save(infile):

    with rasterio.open(infile) as src:

        clipped, transform = mask(
            src, shapes,
            crop=True,
            nodata=src.nodata
        )

        meta = src.meta.copy()

        meta.update({
            "height": clipped.shape[1],
            "width": clipped.shape[2],
            "transform": transform,
            "nodata": src.nodata
        })


        fname = os.path.basename(infile)


        month = get_month(fname)
        var   = get_var(fname)


        new_name = f"T2M_{month}_{var}.tif"

        out_path = os.path.join(out_dir, new_name)


        with rasterio.open(out_path, "w", **meta) as dst:
            dst.write(clipped)


        print("✅", fname, "→", new_name)



# ===============================
# RUN ALL
# ===============================

files = sorted(glob.glob(input_dir + "/*.tif"))

print("📂 Total files:", len(files), "\n")


for f in files:
    clip_rename_save(f)


print("\n🎉 DONE")
print("📁 Output:", out_dir)


In [ ]:
# ============================================
# PROPER CLIP + MASK (NO BACKGROUND)
# T2M SAFE VERSION
# ============================================

import os
import glob
import re
import geopandas as gpd
import rasterio
from rasterio.mask import mask
from shapely.geometry import mapping
import numpy as np


# ===============================
# PATHS
# ===============================

shp_path = "/content/1a_GRB_WGS84.shp"

input_dir = "/content/ERA5_All_Temperature_Trends/ERA5_All_Temperature_Trends/T2M_Trend"

out_dir = "/content/CLIPPED_T2M_RENAMED_V3_NO_BG"

os.makedirs(out_dir, exist_ok=True)


# ===============================
# LOAD SHAPEFILE
# ===============================

gdf = gpd.read_file(shp_path)
shapes = [mapping(geom) for geom in gdf.geometry]


# ===============================
# MONTH MAP
# ===============================

month_map_num = {
    "01":"JAN","02":"FEB","03":"MAR","04":"APR",
    "05":"MAY","06":"JUN","07":"JUL","08":"AUG",
    "09":"SEP","10":"OCT","11":"NOV"
}

month_map_txt = {
    "jan":"JAN","feb":"FEB","mar":"MAR","apr":"APR",
    "may":"MAY","jun":"JUN","jul":"JUL","aug":"AUG",
    "sep":"SEP","oct":"OCT","nov":"NOV","dec":"DEC"
}


# ===============================
# HELPERS
# ===============================

def get_month(fname):

    fname = fname.lower()

    m1 = re.search(r"month_(\d{2})", fname)
    if m1:
        return month_map_num.get(m1.group(1))

    for k in month_map_txt:
        if f"_{k}_" in fname:
            return month_map_txt[k]

    return "UNK"


def get_var(fname):

    fname = fname.lower()

    if "pvalue" in fname:
        return "PVAL"
    if "senslope" in fname:
        return "SEN"
    if "tfpw" in fname:
        return "TFPW"

    return "DATA"


# ===============================
# CLIP FUNCTION (FIXED)
# ===============================

def clip_rename_save(infile):

    with rasterio.open(infile) as src:

        # Force mask outside polygon
        clipped, transform = mask(
            src,
            shapes,
            crop=True,
            filled=True,
            nodata=-9999
        )


        # Replace any leftover values
        clipped = np.where(clipped == src.nodata, -9999, clipped)


        meta = src.meta.copy()

        meta.update({
            "height": clipped.shape[1],
            "width": clipped.shape[2],
            "transform": transform,
            "nodata": -9999,
            "compress": "lzw"
        })


        fname = os.path.basename(infile)

        month = get_month(fname)
        var   = get_var(fname)

        new_name = f"T2M_{month}_{var}.tif"

        out_path = os.path.join(out_dir, new_name)


        with rasterio.open(out_path, "w", **meta) as dst:
            dst.write(clipped)


        print("✅", new_name)



# ===============================
# RUN ALL
# ===============================

files = sorted(glob.glob(input_dir + "/*.tif"))

print("📂 Total files:", len(files), "\n")


for f in files:
    clip_rename_save(f)


print("\n🎉 DONE")
print("📁 Output:", out_dir)


In [ ]:
import shutil
from google.colab import files

# Folder to zip
folder = "/content/CLIPPED_T2M_RENAMED_V3_NO_BG"
zip_path = "/content/CLIPPED_T2M_RENAMED_V3_NO_BG.zip"

# Create zip
shutil.make_archive(zip_path.replace(".zip",""), "zip", folder)

print("✅ ZIP created:", zip_path)

# Download
files.download(zip_path)


In [ ]:
import zipfile
import os

zip_path = "/content/MIVD_mm_per_day.zip"
extract_path = "/content/MIVD_mm_per_day"

# Unzip
with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(extract_path)

print("✅ Unzipped to:", extract_path, "\n")

# Show contents
for root, dirs, files in os.walk(extract_path):
    level = root.replace(extract_path, "").count(os.sep)
    indent = " " * 4 * level
    print(f"{indent}{os.path.basename(root)}/")

    subindent = " " * 4 * (level + 1)
    for f in files:
        print(f"{subindent}{f}")


In [ ]:
# ======================================================================
# CREATE TRUE ANNUAL STACK (1951–2025) FOR MIVD
# 1 BAND = 1 YEAR
# FROM 12 MONTHLY FILES
# ======================================================================

import xarray as xr
import glob
import os


# ----------------------------------------------------------------------
# PATHS
# ----------------------------------------------------------------------

data_dir = "/content/MIVD_mm_per_day"

out_dir  = "/content/MIVD_Annual_75Band"
os.makedirs(out_dir, exist_ok=True)


# ----------------------------------------------------------------------
# 1. READ ALL MONTHLY FILES
# ----------------------------------------------------------------------

files = sorted(glob.glob(f"{data_dir}/MIVD_*.nc"))

print("📂 Total monthly files:", len(files))

datasets = [xr.open_dataset(f) for f in files]

# Merge all months along time
ds = xr.concat(datasets, dim="valid_time")


# ----------------------------------------------------------------------
# 2. CHECK VARIABLE NAME
# ----------------------------------------------------------------------

print("Variables:", list(ds.data_vars))

# Usually variable = MIVD
mivd = ds["MIVD"]


# ----------------------------------------------------------------------
# 3. GROUP BY YEAR → 1 BAND PER YEAR
# ----------------------------------------------------------------------

annual_75 = mivd.groupby(
    mivd.valid_time.dt.year
).mean(dim="valid_time")


# Rename dimension
annual_75 = annual_75.rename({"year": "time"})


# ----------------------------------------------------------------------
# 4. CREATE OUTPUT DATASET
# ----------------------------------------------------------------------

out = xr.Dataset(

    {"MIVD_ANNUAL": annual_75},

    coords={
        "time": sorted(set(mivd.valid_time.dt.year.values)),
        "latitude": mivd.latitude,
        "longitude": mivd.longitude
    }
)


out["MIVD_ANNUAL"].attrs["description"] = \
    "Annual mean MIVD (1 band = 1 year, 1951–2025)"

out["MIVD_ANNUAL"].attrs["units"] = "mm/day"


# ----------------------------------------------------------------------
# 5. SAVE
# ----------------------------------------------------------------------

out_file = f"{out_dir}/MIVD_Annual_Stack_75Band_1951_2025.nc"

out.to_netcdf(out_file)


print("\n✅ Saved:", out_file)
print("📊 TOTAL BANDS:", out.dims["time"])


In [ ]:
# ======================================================================
# MIVD: CREATE 75-BAND ANNUAL STACK + EXPORT TO GEOTIFF
# ======================================================================

import xarray as xr
import glob
import os
import rasterio
from rasterio.transform import from_origin


# ----------------------------------------------------------------------
# PATHS
# ----------------------------------------------------------------------

data_dir = "/content/MIVD_mm_per_day/content/MIVD_mm_per_day"

out_dir  = "/content/MIVD_Annual_TIFF"
os.makedirs(out_dir, exist_ok=True)


# ----------------------------------------------------------------------
# 1. READ ALL MONTHLY FILES
# ----------------------------------------------------------------------

files = sorted(glob.glob(f"{data_dir}/MIVD_*.nc"))

print("📂 Monthly files:", len(files))

datasets = [xr.open_dataset(f) for f in files]

# Combine all months
ds = xr.concat(datasets, dim="valid_time")


# ----------------------------------------------------------------------
# 2. EXTRACT VARIABLE
# ----------------------------------------------------------------------

print("Variables:", list(ds.data_vars))

mivd = ds["MIVD"]   # main variable


# ----------------------------------------------------------------------
# 3. GROUP BY YEAR → ANNUAL MEAN
# ----------------------------------------------------------------------

annual = mivd.groupby(
    mivd.valid_time.dt.year
).mean(dim="valid_time")

# Rename dimension
annual = annual.rename({"year": "time"})


# ----------------------------------------------------------------------
# 4. PREPARE FOR GEOTIFF
# ----------------------------------------------------------------------

data = annual.values   # (time, lat, lon)

n_bands, n_lat, n_lon = data.shape


# Coordinates
lats = annual.latitude.values
lons = annual.longitude.values


# Resolution
lat_res = abs(lats[1] - lats[0])
lon_res = abs(lons[1] - lons[0])


# GeoTransform
transform = from_origin(
    lons.min(),
    lats.max(),
    lon_res,
    lat_res
)


# ----------------------------------------------------------------------
# 5. SAVE MULTIBAND GEOTIFF
# ----------------------------------------------------------------------

out_file = f"{out_dir}/MIVD_Annual_75Band_1951_2025.tif"


with rasterio.open(

    out_file,
    "w",
    driver="GTiff",
    height=n_lat,
    width=n_lon,
    count=n_bands,
    dtype=data.dtype,
    crs="EPSG:4326",
    transform=transform,
    nodata=-9999,
    compress="lzw"

) as dst:

    for i in range(n_bands):

        band_data = data[i]

        band_data = band_data.astype("float32")
        band_data[~(band_data == band_data)] = -9999   # NaN → NoData

        dst.write(band_data, i + 1)


print("\n✅ GeoTIFF Created!")
print("📁 File:", out_file)
print("📊 Bands:", n_bands)


In [ ]:
# ======================================================================
# MVIMD: CREATE 75-BAND ANNUAL STACK + EXPORT TO GEOTIFF
# VAR = avg_vimdf  →  MVIMD_ANN
# ======================================================================

import xarray as xr
import glob
import os
import rasterio
from rasterio.transform import from_origin
import numpy as np


# ----------------------------------------------------------------------
# PATHS
# ----------------------------------------------------------------------

data_dir = "/content/MIVD_mm_per_day/content/MIVD_mm_per_day"

out_dir  = "/content/MVIMD_Annual_TIFF"
os.makedirs(out_dir, exist_ok=True)


# ----------------------------------------------------------------------
# 1. READ ALL MONTHLY FILES
# ----------------------------------------------------------------------

files = sorted(glob.glob(f"{data_dir}/MIVD_*.nc"))

print("📂 Monthly files:", len(files))

datasets = [xr.open_dataset(f) for f in files]

# Combine all months
ds = xr.concat(datasets, dim="valid_time")


# ----------------------------------------------------------------------
# 2. USE CORRECT VARIABLE
# ----------------------------------------------------------------------

print("Variables:", list(ds.data_vars))

mivd = ds["avg_vimdf"]   # ✅ correct name


# ----------------------------------------------------------------------
# 3. GROUP BY YEAR → ANNUAL MEAN
# ----------------------------------------------------------------------

annual = mivd.groupby(
    mivd.valid_time.dt.year
).mean(dim="valid_time")

# Rename dimension
annual = annual.rename({"year": "time"})


# ----------------------------------------------------------------------
# 4. PREPARE FOR GEOTIFF
# ----------------------------------------------------------------------

data = annual.values.astype("float32")   # (time, lat, lon)

n_bands, n_lat, n_lon = data.shape


# Coordinates
lats = annual.latitude.values
lons = annual.longitude.values


# Resolution
lat_res = abs(lats[1] - lats[0])
lon_res = abs(lons[1] - lons[0])


# GeoTransform
transform = from_origin(
    lons.min(),
    lats.max(),
    lon_res,
    lat_res
)


# Replace NaN → NoData
data[np.isnan(data)] = -9999


# ----------------------------------------------------------------------
# 5. SAVE MULTIBAND GEOTIFF
# ----------------------------------------------------------------------

out_file = f"{out_dir}/MVIMD_ANN_75Band_1951_2025.tif"


with rasterio.open(

    out_file,
    "w",
    driver="GTiff",
    height=n_lat,
    width=n_lon,
    count=n_bands,
    dtype="float32",
    crs="EPSG:4326",
    transform=transform,
    nodata=-9999,
    compress="lzw"

) as dst:

    for i in range(n_bands):
        dst.write(data[i], i + 1)


print("\n✅ GeoTIFF Created!")
print("📁 File:", out_file)
print("📊 Bands:", n_bands)


In [ ]:
from google.colab import files

file_path = "/content/MVIMD_Annual_TIFF/MVIMD_ANN_75Band_1951_2025.tif"

files.download(file_path)


In [ ]:
# ======================================================================
# MVIMD SEASONAL STACK (1951–2025) → MULTIBAND GEOTIFF
# Variable: avg_vimdf
# ======================================================================

import xarray as xr
import glob
import os
import rasterio
from rasterio.transform import from_origin
import numpy as np


# ----------------------------------------------------------------------
# PATHS
# ----------------------------------------------------------------------

data_dir = "/content/MIVD_mm_per_day/content/MIVD_mm_per_day"

out_dir  = "/content/MVIMD_Seasonal_TIFF"
os.makedirs(out_dir, exist_ok=True)


# ----------------------------------------------------------------------
# 1. READ ALL FILES
# ----------------------------------------------------------------------

files = sorted(glob.glob(f"{data_dir}/MIVD_*.nc"))

print("📂 Monthly files:", len(files))

datasets = [xr.open_dataset(f) for f in files]

ds = xr.concat(datasets, dim="valid_time")


# ----------------------------------------------------------------------
# 2. SELECT VARIABLE
# ----------------------------------------------------------------------

print("Variables:", list(ds.data_vars))

mivd = ds["avg_vimdf"]


# ----------------------------------------------------------------------
# 3. DEFINE SEASONS
# ----------------------------------------------------------------------

seasons = {

    "WINTER"      : [1, 2],          # Jan–Feb
    "PREMONSOON"  : [3, 4, 5],       # Mar–May
    "MONSOON"     : [6, 7, 8, 9],    # Jun–Sep
    "POSTMONSOON" : [10, 11, 12]     # Oct–Dec
}


# ----------------------------------------------------------------------
# 4. GEO INFO
# ----------------------------------------------------------------------

lats = mivd.latitude.values
lons = mivd.longitude.values

lat_res = abs(lats[1] - lats[0])
lon_res = abs(lons[1] - lons[0])

transform = from_origin(
    lons.min(),
    lats.max(),
    lon_res,
    lat_res
)


# ----------------------------------------------------------------------
# 5. PROCESS EACH SEASON
# ----------------------------------------------------------------------

for season, months in seasons.items():

    print("\n🚀 Processing:", season)


    # --------------------------------------------------------------
    # Select season months
    # --------------------------------------------------------------

    season_data = mivd.sel(
        valid_time = mivd.valid_time.dt.month.isin(months)
    )


    # --------------------------------------------------------------
    # Annual mean inside season
    # --------------------------------------------------------------

    seasonal_annual = season_data.groupby(
        season_data.valid_time.dt.year
    ).mean(dim="valid_time")

    seasonal_annual = seasonal_annual.rename({"year": "time"})


    # --------------------------------------------------------------
    # Prepare data
    # --------------------------------------------------------------

    data = seasonal_annual.values.astype("float32")

    data[np.isnan(data)] = -9999


    n_bands, n_lat, n_lon = data.shape


    # --------------------------------------------------------------
    # Output file
    # --------------------------------------------------------------

    out_file = f"{out_dir}/MVIMD_{season}_75Band_1951_2025.tif"


    # --------------------------------------------------------------
    # Save GeoTIFF
    # --------------------------------------------------------------

    with rasterio.open(

        out_file,
        "w",
        driver="GTiff",
        height=n_lat,
        width=n_lon,
        count=n_bands,
        dtype="float32",
        crs="EPSG:4326",
        transform=transform,
        nodata=-9999,
        compress="lzw"

    ) as dst:

        for i in range(n_bands):
            dst.write(data[i], i + 1)


    print("✅ Saved:", out_file)
    print("📊 Bands:", n_bands)


# ----------------------------------------------------------------------
# 6. DONE
# ----------------------------------------------------------------------

print("\n🎉 All seasonal MVIMD GeoTIFFs created successfully!")
print("📁 Output folder:", out_dir)


In [ ]:
import shutil
from google.colab import files

# Folder to zip
folder = "/content/MVIMD_Seasonal_TIFF"
zip_path = "/content/MVIMD_Seasonal_TIFF_ALL.zip"

# Create ZIP
shutil.make_archive(zip_path.replace(".zip",""), "zip", folder)

print("✅ ZIP created:", zip_path)

# Download
files.download(zip_path)


FileNotFoundError: [Errno 2] No such file or directory: '/content/MVIMD_Seasonal_TIFF'

In [ ]:
import xarray as xr
import glob
import numpy as np
import os


# ======================================
# Input / Output
# ======================================

data_dir = "/content/MIVD_mm_per_day/content/MIVD_mm_per_day"

out_dir  = "/content/MVIMD_Seasonal_75Band_SAFE"
os.makedirs(out_dir, exist_ok=True)


# ======================================
# Load monthly files
# ======================================

files = sorted(glob.glob(f"{data_dir}/MIVD_*.nc"))

print("📂 Files:", len(files))

datasets = [xr.open_dataset(f) for f in files]

ds = xr.concat(datasets, dim="valid_time")


# ======================================
# Select variable
# ======================================

print("Variables:", list(ds.data_vars))

mivd = ds["avg_vimdf"]   # correct variable


# ======================================
# Define seasons
# ======================================

seasons = {

    "WINTER"      : [1, 2],          # Jan–Feb
    "PREMONSOON"  : [3, 4, 5],       # Mar–May
    "MONSOON"     : [6, 7, 8, 9],    # Jun–Sep
    "POSTMONSOON" : [10, 11, 12]     # Oct–Dec
}


# ======================================
# CRS (EPSG:4326)
# ======================================

crs = xr.DataArray(

    0,

    attrs={
        "grid_mapping_name": "latitude_longitude",
        "epsg_code": "EPSG:4326",
        "semi_major_axis": 6378137.0,
        "inverse_flattening": 298.257223563,
        "longitude_of_prime_meridian": 0.0,
        "spatial_ref":
        "GEOGCS[\"WGS 84\",DATUM[\"WGS_1984\","
        "SPHEROID[\"WGS 84\",6378137,298.257223563]],"
        "PRIMEM[\"Greenwich\",0],"
        "UNIT[\"degree\",0.0174532925199433]]"
    },

    name="crs"
)


# ======================================
# Process each season
# ======================================

for season, months in seasons.items():

    print("\n🚀 Processing:", season)


    # ------------------------------
    # Select season
    # ------------------------------

    season_data = mivd.sel(
        valid_time = mivd.valid_time.dt.month.isin(months)
    )


    # ------------------------------
    # Annual mean
    # ------------------------------

    season_year = season_data.groupby(
        "valid_time.year"
    ).mean("valid_time")


    # ------------------------------
    # Year → valid_time
    # ------------------------------

    years = season_year.year.values

    mid_month = months[len(months)//2]

    time = np.array([
        np.datetime64(f"{y}-{mid_month:02d}-01") for y in years
    ])


    season_year = season_year.assign_coords(
        valid_time=("year", time)
    )

    season_year = season_year.swap_dims(
        {"year": "valid_time"}
    )

    season_year = season_year.drop_vars("year")


    # ------------------------------
    # Break HDF lock
    # ------------------------------

    season_np = np.array(season_year.values)


    clean_da = xr.DataArray(

        season_np,

        dims=("valid_time", "latitude", "longitude"),

        coords={
            "valid_time": season_year.valid_time.values,
            "latitude": season_year.latitude.values,
            "longitude": season_year.longitude.values
        },

        name=f"MVIMD_{season}"
    )


    # ------------------------------
    # Add CRS
    # ------------------------------

    clean_da.attrs["grid_mapping"] = "crs"


    # ------------------------------
    # Output Dataset
    # ------------------------------

    ds_out = xr.Dataset(
        {
            f"MVIMD_{season}": clean_da,
            "crs": crs
        }
    )


    # ------------------------------
    # Save
    # ------------------------------

    out_file = f"{out_dir}/MVIMD_{season}_75Band_1951_2025.nc"


    if os.path.exists(out_file):
        os.remove(out_file)


    ds_out.to_netcdf(out_file, engine="scipy")


    print("✅ Saved:", out_file)
    print("📊 Bands:", clean_da.shape[0])


# ======================================
# Done
# ======================================

print("\n🎉 All seasonal MVIMD stacks created safely!")
print("📁 Output:", out_dir)


In [ ]:
import shutil
from google.colab import files

# Folder to zip
folder = "/content/MVIMD_Seasonal_75Band_SAFE"
zip_path = "/content/MVIMD_Seasonal_75Band_SAFE.zip"

# Create ZIP
shutil.make_archive(zip_path.replace(".zip",""), "zip", folder)

print("✅ ZIP created:", zip_path)

# Download
files.download(zip_path)


In [ ]:
import xarray as xr
import glob
import os


# ======================================
# Paths
# ======================================

input_dir = "/content/MVIMD_Seasonal_75Band_SAFE"
out_dir   = "/content/MVIMD_Seasonal_75Band_FIXED_CRS"

os.makedirs(out_dir, exist_ok=True)


# ======================================
# Process all seasonal files
# ======================================

files = sorted(glob.glob(f"{input_dir}/*.nc"))

print("📂 Files found:", len(files))


for f in files:

    print("\nFixing:", os.path.basename(f))


    # Load dataset
    ds = xr.open_dataset(f)


    # ----------------------------------
    # Set lat/lon as spatial coordinates
    # ----------------------------------

    ds = ds.set_coords(["latitude", "longitude"])


    # ----------------------------------
    # Add CF-compliant CRS
    # ----------------------------------

    ds["crs"] = xr.DataArray(
        0,
        attrs={
            "grid_mapping_name": "latitude_longitude",
            "epsg_code": "EPSG:4326",
            "semi_major_axis": 6378137.0,
            "inverse_flattening": 298.257223563,
            "longitude_of_prime_meridian": 0.0,
            "spatial_ref":
            "GEOGCS[\"WGS 84\",DATUM[\"WGS_1984\","
            "SPHEROID[\"WGS 84\",6378137,298.257223563]],"
            "PRIMEM[\"Greenwich\",0],"
            "UNIT[\"degree\",0.0174532925199433]]"
        }
    )


    # ----------------------------------
    # Link CRS to main variable
    # ----------------------------------

    main_var = list(ds.data_vars)[0]

    ds[main_var].attrs["grid_mapping"] = "crs"


    # ----------------------------------
    # Global projection info
    # ----------------------------------

    ds.attrs["Conventions"] = "CF-1.8"
    ds.attrs["geospatial_lat_min"] = float(ds.latitude.min())
    ds.attrs["geospatial_lat_max"] = float(ds.latitude.max())
    ds.attrs["geospatial_lon_min"] = float(ds.longitude.min())
    ds.attrs["geospatial_lon_max"] = float(ds.longitude.max())


    # ----------------------------------
    # Save fixed file
    # ----------------------------------

    out_file = os.path.join(
        out_dir,
        os.path.basename(f).replace(".nc", "_CRS_FIXED.nc")
    )


    if os.path.exists(out_file):
        os.remove(out_file)


    ds.to_netcdf(out_file, engine="scipy")

    print("✅ Saved:", out_file)


print("\n🎉 All files fixed with proper CRS!")
print("📁 Output:", out_dir)


In [ ]:
# ======================================================================
# CONVERT MVIMD SEASONAL NETCDF → MULTIBAND GEOTIFF (WITH CRS)
# ======================================================================

import xarray as xr
import glob
import os
import rasterio
from rasterio.transform import from_origin
import numpy as np


# ====================================================
# INPUT / OUTPUT
# ====================================================

input_dir = "/content/MVIMD_Seasonal_75Band_FIXED_CRS"

out_dir   = "/content/MVIMD_Seasonal_MultiBand_TIFF"
os.makedirs(out_dir, exist_ok=True)


# ====================================================
# READ FILES
# ====================================================

files = sorted(glob.glob(f"{input_dir}/*.nc"))

print("📂 NetCDF files:", len(files))


# ====================================================
# PROCESS EACH FILE
# ====================================================

for f in files:

    print("\n🚀 Processing:", os.path.basename(f))


    # -----------------------------------------------
    # Open dataset
    # -----------------------------------------------

    ds = xr.open_dataset(f)


    # Get main variable
    var_name = list(ds.data_vars)[0]

    da = ds[var_name]


    # -----------------------------------------------
    # Extract data
    # -----------------------------------------------

    data = da.values.astype("float32")

    data[np.isnan(data)] = -9999


    n_bands, n_lat, n_lon = data.shape


    # -----------------------------------------------
    # Coordinates
    # -----------------------------------------------

    lats = da.latitude.values
    lons = da.longitude.values


    lat_res = abs(lats[1] - lats[0])
    lon_res = abs(lons[1] - lons[0])


    # GeoTransform
    transform = from_origin(
        lons.min(),
        lats.max(),
        lon_res,
        lat_res
    )


    # -----------------------------------------------
    # Output filename
    # -----------------------------------------------

    base = os.path.basename(f).replace("_CRS_FIXED.nc", "")

    out_file = f"{out_dir}/{base}_MULTIBAND.tif"


    # -----------------------------------------------
    # Write GeoTIFF
    # -----------------------------------------------

    with rasterio.open(

        out_file,
        "w",
        driver="GTiff",
        height=n_lat,
        width=n_lon,
        count=n_bands,
        dtype="float32",
        crs="EPSG:4326",
        transform=transform,
        nodata=-9999,
        compress="lzw"

    ) as dst:

        for i in range(n_bands):
            dst.write(data[i], i + 1)


    print("✅ Saved:", out_file)
    print("📊 Bands:", n_bands)


# ====================================================
# DONE
# ====================================================

print("\n🎉 All NetCDF files converted to Multi-band GeoTIFF!")
print("📁 Output folder:", out_dir)


In [ ]:
import shutil
from google.colab import files

# Folder to zip
folder = "/content/MVIMD_Seasonal_MultiBand_TIFF"
zip_path = "/content/MVIMD_Seasonal_MultiBand_TIFF.zip"

# Create ZIP
shutil.make_archive(zip_path.replace(".zip",""), "zip", folder)

print("✅ ZIP created:", zip_path)

# Download
files.download(zip_path)


In [ ]:
# ======================================================================
# CALCULATE MEAN OF SEASONAL MULTIBAND MVIMD TIFFs
# ======================================================================

import rasterio
import glob
import os
import numpy as np


# ====================================================
# INPUT / OUTPUT
# ====================================================

input_dir = "/content/MVIMD_Seasonal_MultiBand_TIFF"

out_dir   = "/content/MVIMD_Seasonal_MEAN"
os.makedirs(out_dir, exist_ok=True)


# ====================================================
# READ ALL TIFF FILES
# ====================================================

files = sorted(glob.glob(f"{input_dir}/*.tif"))

print("📂 Files found:", len(files))


# ====================================================
# PROCESS EACH FILE
# ====================================================

for f in files:

    name = os.path.basename(f)

    print("\n🚀 Processing:", name)


    # -----------------------------------------------
    # Open raster
    # -----------------------------------------------

    with rasterio.open(f) as src:

        data = src.read().astype("float32")   # (bands, rows, cols)

        profile = src.profile


    # -----------------------------------------------
    # Replace NoData with NaN
    # -----------------------------------------------

    nodata = profile.get("nodata", -9999)

    data[data == nodata] = np.nan


    # -----------------------------------------------
    # Mean across bands
    # -----------------------------------------------

    mean_data = np.nanmean(data, axis=0)   # (rows, cols)


    # -----------------------------------------------
    # Output name
    # -----------------------------------------------

    if "WINTER" in name.upper():
        season = "WINTER"

    elif "PREMONSOON" in name.upper():
        season = "PREMONSOON"

    elif "MONSOON" in name.upper():
        season = "MONSOON"

    elif "POSTMONSOON" in name.upper():
        season = "POSTMONSOON"

    else:
        season = "UNKNOWN"


    out_name = f"MVIMD_{season}_MEAN.tif"

    out_file = os.path.join(out_dir, out_name)


    # -----------------------------------------------
    # Update profile
    # -----------------------------------------------

    profile.update(
        count=1,
        dtype="float32",
        nodata=-9999,
        compress="lzw"
    )


    # Replace NaN → NoData
    mean_data[np.isnan(mean_data)] = -9999


    # -----------------------------------------------
    # Save
    # -----------------------------------------------

    with rasterio.open(out_file, "w", **profile) as dst:

        dst.write(mean_data, 1)


    print("✅ Saved:", out_file)


# ====================================================
# DONE
# ====================================================

print("\n🎉 All seasonal mean maps created!")
print("📁 Output folder:", out_dir)


In [ ]:
import shutil
from google.colab import files

# Folder to zip
folder = "/content/MVIMD_Seasonal_MEAN"
zip_path = "/content/MVIMD_Seasonal_MEAN.zip"

# Create ZIP
shutil.make_archive(zip_path.replace(".zip",""), "zip", folder)

print("✅ ZIP created:", zip_path)

# Download
files.download(zip_path)


In [ ]:
# ======================================================================
# CALCULATE MEAN OF ALL SEASONAL MVIMD MULTIBAND TIFFs (FIXED)
# ======================================================================

import rasterio
import glob
import os
import numpy as np


# ====================================================
# INPUT / OUTPUT
# ====================================================

input_dir = "/content/MVIMD_Seasonal_MultiBand_TIFF"

out_dir   = "/content/MVIMD_Seasonal_MEAN_V2"
os.makedirs(out_dir, exist_ok=True)


# ====================================================
# READ FILES
# ====================================================

files = sorted(glob.glob(f"{input_dir}/*.tif"))

print("📂 Files found:", len(files))
print(files)


# ====================================================
# PROCESS EACH FILE
# ====================================================

for f in files:

    fname = os.path.basename(f).upper()

    print("\n🚀 Processing:", fname)


    # -----------------------------------------------
    # Detect season (ROBUST)
    # -----------------------------------------------

    if "WINTER" in fname:
        season = "WINTER"

    elif "PRE" in fname and "MON" not in fname:
        season = "PREMONSOON"

    elif "PREMONSOON" in fname:
        season = "PREMONSOON"

    elif "MONSOON" in fname and "POST" not in fname:
        season = "MONSOON"

    elif "POST" in fname:
        season = "POSTMONSOON"

    else:
        season = "UNKNOWN"


    print("➡️ Detected season:", season)


    # -----------------------------------------------
    # Open raster
    # -----------------------------------------------

    with rasterio.open(f) as src:

        data = src.read().astype("float32")

        profile = src.profile


    # -----------------------------------------------
    # Handle NoData
    # -----------------------------------------------

    nodata = profile.get("nodata", -9999)

    data[data == nodata] = np.nan


    # -----------------------------------------------
    # Mean
    # -----------------------------------------------

    mean_data = np.nanmean(data, axis=0)


    # -----------------------------------------------
    # Output name
    # -----------------------------------------------

    out_name = f"MVIMD_{season}_MEAN_1951_2025.tif"

    out_file = os.path.join(out_dir, out_name)


    # -----------------------------------------------
    # Update profile
    # -----------------------------------------------

    profile.update(
        count=1,
        dtype="float32",
        nodata=-9999,
        compress="lzw"
    )


    mean_data[np.isnan(mean_data)] = -9999


    # -----------------------------------------------
    # Save
    # -----------------------------------------------

    with rasterio.open(out_file, "w", **profile) as dst:

        dst.write(mean_data, 1)


    print("✅ Saved:", out_file)


# ====================================================
# DONE
# ====================================================

print("\n🎉 All seasonal mean maps created correctly!")
print("📁 Output folder:", out_dir)


In [ ]:
import shutil
from google.colab import files

# Folder to zip
folder = "/content/MVIMD_Seasonal_MEAN_V2"
zip_path = "/content/MVIMD_Seasonal_MEAN_V2.zip"

# Create ZIP
shutil.make_archive(zip_path.replace(".zip",""), "zip", folder)

print("✅ ZIP created:", zip_path)

# Download
files.download(zip_path)


In [ ]:
# ======================================================================
# CREATE MEAN MAP FROM MVIMD ANNUAL 75-BAND TIFF
# ======================================================================

import rasterio
import numpy as np
import os


# ====================================================
# INPUT / OUTPUT
# ====================================================

input_file = "/content/MVIMD_ANN_75Band_1951_2025.tif"

out_dir = "/content/MVIMD_Annual_MEAN"
os.makedirs(out_dir, exist_ok=True)

out_file = f"{out_dir}/MVIMD_ANN_MEAN_1951_2025.tif"


# ====================================================
# READ RASTER
# ====================================================

with rasterio.open(input_file) as src:

    data = src.read().astype("float32")   # (bands, rows, cols)

    profile = src.profile

    nodata = src.nodata


print("📊 Total bands:", data.shape[0])


# ====================================================
# HANDLE NODATA
# ====================================================

if nodata is None:
    nodata = -9999

data[data == nodata] = np.nan


# ====================================================
# CALCULATE MEAN
# ====================================================

mean_data = np.nanmean(data, axis=0)


# ====================================================
# PREPARE OUTPUT PROFILE
# ====================================================

profile.update(
    count=1,
    dtype="float32",
    nodata=-9999,
    compress="lzw"
)


# Replace NaN → NoData
mean_data[np.isnan(mean_data)] = -9999


# ====================================================
# SAVE
# ====================================================

with rasterio.open(out_file, "w", **profile) as dst:

    dst.write(mean_data, 1)


print("\n✅ Annual mean file created!")
print("📁 File:", out_file)


In [ ]:
from google.colab import files

file_path = "/content/MVIMD_Annual_MEAN/MVIMD_ANN_MEAN_1951_2025.tif"

files.download(file_path)


In [ ]:
import shutil
from google.colab import files

# Folder to zip
folder = "/content/CLIPPED_T2M_RENAMED_V2"
zip_path = "/content/CLIPPED_T2M_RENAMED_V2.zip"

# Create zip
shutil.make_archive(zip_path.replace(".zip",""), "zip", folder)

print("✅ ZIP created:", zip_path)

# Download
files.download(zip_path)


In [3]:
import zipfile
import os

zip_path = "/content/TIFF_MVIMD.zip"
extract_path = "/content/TIFF_MVIMD"

# Create folder
os.makedirs(extract_path, exist_ok=True)

# Unzip
with zipfile.ZipFile(zip_path, 'r') as z:
    z.extractall(extract_path)

print("✅ UNZIPPED TO:", extract_path)
print("\n📂 FILES FOUND:\n")

# Show all files
for root, dirs, files in os.walk(extract_path):
    for f in files:
        print(os.path.join(root, f))


✅ UNZIPPED TO: /content/TIFF_MVIMD

📂 FILES FOUND:

/content/TIFF_MVIMD/TIFF_MVIMD/ANNUAL/MVIMD_ANN_MEAN_1951_2025.tif
/content/TIFF_MVIMD/TIFF_MVIMD/SEASONAL/MVIMD_Seasonal_MEAN_V2/MVIMD_PREMONSOON_MEAN_1951_2025.tif
/content/TIFF_MVIMD/TIFF_MVIMD/SEASONAL/MVIMD_Seasonal_MEAN_V2/MVIMD_WINTER_MEAN_1951_2025.tif
/content/TIFF_MVIMD/TIFF_MVIMD/SEASONAL/MVIMD_Seasonal_MEAN_V2/MVIMD_POSTMONSOON_MEAN_1951_2025.tif
/content/TIFF_MVIMD/TIFF_MVIMD/SEASONAL/MVIMD_Seasonal_MEAN_V2/MVIMD_MONSOON_MEAN_1951_2025.tif


In [4]:
import os
import glob
import geopandas as gpd
import rasterio
from rasterio.mask import mask

# ===============================
# PATHS
# ===============================

shp_path = "/content/1a_GRB_WGS84.shp"

input_files = [
    "/content/TIFF_MVIMD/TIFF_MVIMD/ANNUAL/MVIMD_ANN_MEAN_1951_2025.tif",
    "/content/TIFF_MVIMD/TIFF_MVIMD/SEASONAL/MVIMD_Seasonal_MEAN_V2/MVIMD_PREMONSOON_MEAN_1951_2025.tif",
    "/content/TIFF_MVIMD/TIFF_MVIMD/SEASONAL/MVIMD_Seasonal_MEAN_V2/MVIMD_WINTER_MEAN_1951_2025.tif",
    "/content/TIFF_MVIMD/TIFF_MVIMD/SEASONAL/MVIMD_Seasonal_MEAN_V2/MVIMD_POSTMONSOON_MEAN_1951_2025.tif",
    "/content/TIFF_MVIMD/TIFF_MVIMD/SEASONAL/MVIMD_Seasonal_MEAN_V2/MVIMD_MONSOON_MEAN_1951_2025.tif"
]

output_dir = "/content/MVIMD_CLIPPED_NO_BG"
os.makedirs(output_dir, exist_ok=True)

# ===============================
# READ SHAPEFILE
# ===============================

gdf = gpd.read_file(shp_path)
shapes = gdf.geometry.values


# ===============================
# CLIP FUNCTION
# ===============================

def clip_raster(input_tif, output_tif):

    with rasterio.open(input_tif) as src:

        clipped, transform = mask(
            src,
            shapes,
            crop=True,
            nodata=src.nodata,
            filled=True
        )

        meta = src.meta.copy()

        meta.update({
            "driver": "GTiff",
            "height": clipped.shape[1],
            "width": clipped.shape[2],
            "transform": transform,
            "nodata": src.nodata
        })

        with rasterio.open(output_tif, "w", **meta) as dst:
            dst.write(clipped)


# ===============================
# PROCESS ALL FILES
# ===============================

print("🚀 STARTING CLIPPING...\n")

for tif in input_files:

    name = os.path.basename(tif).replace(".tif", "_CLIPPED.tif")
    out_file = os.path.join(output_dir, name)

    clip_raster(tif, out_file)

    print("✅ DONE:", out_file)


print("\n🎉 ALL FILES CLIPPED SUCCESSFULLY!")
print("📂 OUTPUT FOLDER:", output_dir)


🚀 STARTING CLIPPING...

✅ DONE: /content/MVIMD_CLIPPED_NO_BG/MVIMD_ANN_MEAN_1951_2025_CLIPPED.tif
✅ DONE: /content/MVIMD_CLIPPED_NO_BG/MVIMD_PREMONSOON_MEAN_1951_2025_CLIPPED.tif
✅ DONE: /content/MVIMD_CLIPPED_NO_BG/MVIMD_WINTER_MEAN_1951_2025_CLIPPED.tif
✅ DONE: /content/MVIMD_CLIPPED_NO_BG/MVIMD_POSTMONSOON_MEAN_1951_2025_CLIPPED.tif
✅ DONE: /content/MVIMD_CLIPPED_NO_BG/MVIMD_MONSOON_MEAN_1951_2025_CLIPPED.tif

🎉 ALL FILES CLIPPED SUCCESSFULLY!
📂 OUTPUT FOLDER: /content/MVIMD_CLIPPED_NO_BG


In [5]:
import os
import zipfile

folder_path = "/content/MVIMD_CLIPPED_NO_BG"
zip_path = "/content/MVIMD_CLIPPED_NO_BG.zip"

# Create ZIP
with zipfile.ZipFile(zip_path, "w", zipfile.ZIP_DEFLATED) as z:
    for file in os.listdir(folder_path):
        if file.endswith(".tif"):
            z.write(
                os.path.join(folder_path, file),
                arcname=file
            )

print("✅ ZIP CREATED:", zip_path)
print("📥 READY TO DOWNLOAD")


✅ ZIP CREATED: /content/MVIMD_CLIPPED_NO_BG.zip
📥 READY TO DOWNLOAD


In [6]:
import zipfile
import os

zip_path = "/content/MIVD_Monthly_MEAN_TIFF.zip"
extract_path = "/content/MIVD_Monthly_MEAN_TIFF"

# Create folder
os.makedirs(extract_path, exist_ok=True)

# Unzip
with zipfile.ZipFile(zip_path, "r") as z:
    z.extractall(extract_path)

print("✅ UNZIPPED TO:", extract_path)
print("\n📂 FILES FOUND:\n")

# Show files
for root, dirs, files in os.walk(extract_path):
    for f in files:
        print(os.path.join(root, f))


✅ UNZIPPED TO: /content/MIVD_Monthly_MEAN_TIFF

📂 FILES FOUND:

/content/MIVD_Monthly_MEAN_TIFF/MIVD_APR_MMDAY_MEAN_1951_2025.tif
/content/MIVD_Monthly_MEAN_TIFF/MIVD_JUL_MMDAY_MEAN_1951_2025.tif
/content/MIVD_Monthly_MEAN_TIFF/MIVD_FEB_MMDAY_MEAN_1951_2025.tif
/content/MIVD_Monthly_MEAN_TIFF/MIVD_JAN_MMDAY_MEAN_1951_2025.tif
/content/MIVD_Monthly_MEAN_TIFF/MIVD_JUN_MMDAY_MEAN_1951_2025.tif
/content/MIVD_Monthly_MEAN_TIFF/MIVD_MAR_MMDAY_MEAN_1951_2025.tif
/content/MIVD_Monthly_MEAN_TIFF/MIVD_DEC_MMDAY_MEAN_1951_2025.tif
/content/MIVD_Monthly_MEAN_TIFF/MIVD_SEP_MMDAY_MEAN_1951_2025.tif
/content/MIVD_Monthly_MEAN_TIFF/MIVD_AUG_MMDAY_MEAN_1951_2025.tif
/content/MIVD_Monthly_MEAN_TIFF/MIVD_MAY_MMDAY_MEAN_1951_2025.tif
/content/MIVD_Monthly_MEAN_TIFF/MIVD_NOV_MMDAY_MEAN_1951_2025.tif
/content/MIVD_Monthly_MEAN_TIFF/MIVD_OCT_MMDAY_MEAN_1951_2025.tif


In [7]:
import os
import glob
import geopandas as gpd
import rasterio
from rasterio.mask import mask

# ===============================
# PATHS
# ===============================

shp_path = "/content/1a_GRB_WGS84.shp"

input_dir = "/content/MIVD_Monthly_MEAN_TIFF"

output_dir = "/content/MVIMD_MONTHLY_CLIPPED_RENAMED"
os.makedirs(output_dir, exist_ok=True)


# ===============================
# READ SHAPEFILE
# ===============================

gdf = gpd.read_file(shp_path)
shapes = gdf.geometry.values


# ===============================
# MONTH MAP
# ===============================

month_map = {
    "JAN":"JAN", "FEB":"FEB", "MAR":"MAR", "APR":"APR",
    "MAY":"MAY", "JUN":"JUN", "JUL":"JUL", "AUG":"AUG",
    "SEP":"SEP", "OCT":"OCT", "NOV":"NOV", "DEC":"DEC"
}


# ===============================
# CLIP FUNCTION
# ===============================

def clip_raster(input_tif, output_tif):

    with rasterio.open(input_tif) as src:

        clipped, transform = mask(
            src,
            shapes,
            crop=True,
            nodata=src.nodata,
            filled=True
        )

        meta = src.meta.copy()

        meta.update({
            "driver": "GTiff",
            "height": clipped.shape[1],
            "width": clipped.shape[2],
            "transform": transform,
            "nodata": src.nodata
        })

        with rasterio.open(output_tif, "w", **meta) as dst:
            dst.write(clipped)


# ===============================
# PROCESS FILES
# ===============================

print("🚀 CLIPPING + RENAMING...\n")

tif_files = sorted(glob.glob(os.path.join(input_dir, "*.tif")))

for tif in tif_files:

    fname = os.path.basename(tif)

    # Extract month (JAN, FEB, etc.)
    month = fname.split("_")[1]

    new_name = f"MVIMD_{month}_MEAN_CLIPPED.tif"

    out_file = os.path.join(output_dir, new_name)

    clip_raster(tif, out_file)

    print("✅ DONE:", new_name)


print("\n🎉 ALL FILES CLIPPED & RENAMED!")
print("📂 OUTPUT:", output_dir)


🚀 CLIPPING + RENAMING...

✅ DONE: MVIMD_APR_MEAN_CLIPPED.tif
✅ DONE: MVIMD_AUG_MEAN_CLIPPED.tif
✅ DONE: MVIMD_DEC_MEAN_CLIPPED.tif
✅ DONE: MVIMD_FEB_MEAN_CLIPPED.tif
✅ DONE: MVIMD_JAN_MEAN_CLIPPED.tif
✅ DONE: MVIMD_JUL_MEAN_CLIPPED.tif
✅ DONE: MVIMD_JUN_MEAN_CLIPPED.tif
✅ DONE: MVIMD_MAR_MEAN_CLIPPED.tif
✅ DONE: MVIMD_MAY_MEAN_CLIPPED.tif
✅ DONE: MVIMD_NOV_MEAN_CLIPPED.tif
✅ DONE: MVIMD_OCT_MEAN_CLIPPED.tif
✅ DONE: MVIMD_SEP_MEAN_CLIPPED.tif

🎉 ALL FILES CLIPPED & RENAMED!
📂 OUTPUT: /content/MVIMD_MONTHLY_CLIPPED_RENAMED


In [8]:
import os
import zipfile

folder_path = "/content/MVIMD_MONTHLY_CLIPPED_RENAMED"
zip_path = "/content/MVIMD_MONTHLY_CLIPPED_RENAMED.zip"

# Create ZIP file
with zipfile.ZipFile(zip_path, "w", zipfile.ZIP_DEFLATED) as z:
    for file in os.listdir(folder_path):
        if file.endswith(".tif"):
            z.write(
                os.path.join(folder_path, file),
                arcname=file
            )

print("✅ ZIP CREATED:", zip_path)
print("📥 READY TO DOWNLOAD")


✅ ZIP CREATED: /content/MVIMD_MONTHLY_CLIPPED_RENAMED.zip
📥 READY TO DOWNLOAD


In [9]:
from google.colab import files
files.download("/content/MVIMD_MONTHLY_CLIPPED_RENAMED.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [10]:
import zipfile
import os

zip_path = "/content/MVIMD_Seasonal_TIFF_ALL.zip"
extract_path = "/content/MVIMD_Seasonal_TIFF_ALL"

# Create folder
os.makedirs(extract_path, exist_ok=True)

# Unzip
with zipfile.ZipFile(zip_path, "r") as z:
    z.extractall(extract_path)

print("✅ UNZIPPED TO:", extract_path)
print("\n📂 FILES FOUND:\n")

# Show files
for root, dirs, files in os.walk(extract_path):
    for f in files:
        print(os.path.join(root, f))


✅ UNZIPPED TO: /content/MVIMD_Seasonal_TIFF_ALL

📂 FILES FOUND:

/content/MVIMD_Seasonal_TIFF_ALL/MVIMD_PREMONSOON_75Band_1951_2025.tif
/content/MVIMD_Seasonal_TIFF_ALL/MVIMD_MONSOON_75Band_1951_2025.tif
/content/MVIMD_Seasonal_TIFF_ALL/MVIMD_WINTER_75Band_1951_2025.tif
/content/MVIMD_Seasonal_TIFF_ALL/MVIMD_POSTMONSOON_75Band_1951_2025.tif


In [11]:
import os
import glob
import numpy as np
import rasterio

# ===============================
# PATHS
# ===============================

input_dir = "/content/MVIMD_Seasonal_TIFF_ALL"
output_dir = "/content/MVIMD_SEASONAL_SD"

os.makedirs(output_dir, exist_ok=True)


# ===============================
# PROCESS FILES
# ===============================

print("🚀 CALCULATING STANDARD DEVIATION...\n")

tif_files = glob.glob(os.path.join(input_dir, "*.tif"))

for tif in tif_files:

    with rasterio.open(tif) as src:

        data = src.read()   # Shape: (bands, rows, cols)

        # Calculate SD along band axis
        sd_data = np.nanstd(data, axis=0)

        meta = src.meta.copy()

        # Update for single band output
        meta.update({
            "count": 1,
            "dtype": "float32"
        })

        # Output name
        base = os.path.basename(tif).replace(".tif", "")
        out_name = base + "_SD.tif"
        out_path = os.path.join(output_dir, out_name)

        # Save
        with rasterio.open(out_path, "w", **meta) as dst:
            dst.write(sd_data.astype("float32"), 1)

        print("✅ DONE:", out_name)


print("\n🎉 ALL SEASONAL SD FILES CREATED!")
print("📂 OUTPUT FOLDER:", output_dir)


🚀 CALCULATING STANDARD DEVIATION...

✅ DONE: MVIMD_PREMONSOON_75Band_1951_2025_SD.tif
✅ DONE: MVIMD_MONSOON_75Band_1951_2025_SD.tif
✅ DONE: MVIMD_WINTER_75Band_1951_2025_SD.tif
✅ DONE: MVIMD_POSTMONSOON_75Band_1951_2025_SD.tif

🎉 ALL SEASONAL SD FILES CREATED!
📂 OUTPUT FOLDER: /content/MVIMD_SEASONAL_SD


In [12]:
import os
import glob
import numpy as np
import geopandas as gpd
import rasterio
from rasterio.mask import mask

# ===============================
# PATHS
# ===============================

input_dir = "/content/MVIMD_Seasonal_TIFF_ALL"
shp_path = "/content/1a_GRB_WGS84.shp"

output_dir = "/content/MVIMD_SEASONAL_SD_CLIPPED_NO_BG"
os.makedirs(output_dir, exist_ok=True)


# ===============================
# READ SHAPEFILE
# ===============================

gdf = gpd.read_file(shp_path)
shapes = gdf.geometry.values


# ===============================
# PROCESS FILES
# ===============================

print("🚀 CALCULATING SD + CLIPPING...\n")

tif_files = glob.glob(os.path.join(input_dir, "*.tif"))

for tif in tif_files:

    with rasterio.open(tif) as src:

        data = src.read()   # (bands, rows, cols)

        # SD across bands
        sd_data = np.nanstd(data, axis=0)

        meta = src.meta.copy()

        meta.update({
            "count": 1,
            "dtype": "float32"
        })

        # Create temp in-memory SD raster
        temp_meta = meta.copy()

        # Write SD to memory file
        with rasterio.io.MemoryFile() as memfile:
            with memfile.open(**temp_meta) as tmp:

                tmp.write(sd_data.astype("float32"), 1)

                # Clip with shapefile
                clipped, transform = mask(
                    tmp,
                    shapes,
                    crop=True,
                    nodata=src.nodata,
                    filled=True
                )

                out_meta = tmp.meta.copy()

                out_meta.update({
                    "height": clipped.shape[1],
                    "width": clipped.shape[2],
                    "transform": transform,
                    "nodata": src.nodata
                })

                # Output name
                base = os.path.basename(tif).replace(".tif", "")
                out_name = base + "_SD_CLIPPED.tif"
                out_path = os.path.join(output_dir, out_name)

                # Save final file
                with rasterio.open(out_path, "w", **out_meta) as dst:
                    dst.write(clipped)

        print("✅ DONE:", out_name)


print("\n🎉 ALL SD FILES CALCULATED & CLIPPED!")
print("📂 OUTPUT FOLDER:", output_dir)


🚀 CALCULATING SD + CLIPPING...

✅ DONE: MVIMD_PREMONSOON_75Band_1951_2025_SD_CLIPPED.tif
✅ DONE: MVIMD_MONSOON_75Band_1951_2025_SD_CLIPPED.tif
✅ DONE: MVIMD_WINTER_75Band_1951_2025_SD_CLIPPED.tif
✅ DONE: MVIMD_POSTMONSOON_75Band_1951_2025_SD_CLIPPED.tif

🎉 ALL SD FILES CALCULATED & CLIPPED!
📂 OUTPUT FOLDER: /content/MVIMD_SEASONAL_SD_CLIPPED_NO_BG


In [13]:
import os
import zipfile

folder_path = "/content/MVIMD_SEASONAL_SD_CLIPPED_NO_BG"
zip_path = "/content/MVIMD_SEASONAL_SD_CLIPPED_NO_BG.zip"

# Create ZIP
with zipfile.ZipFile(zip_path, "w", zipfile.ZIP_DEFLATED) as z:
    for file in os.listdir(folder_path):
        if file.endswith(".tif"):
            z.write(
                os.path.join(folder_path, file),
                arcname=file
            )

print("✅ ZIP CREATED:", zip_path)
print("📥 READY TO DOWNLOAD")
from google.colab import files
files.download("/content/MVIMD_SEASONAL_SD_CLIPPED_NO_BG.zip")


✅ ZIP CREATED: /content/MVIMD_SEASONAL_SD_CLIPPED_NO_BG.zip
📥 READY TO DOWNLOAD


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [14]:
import os
import geopandas as gpd
import rasterio
from rasterio.mask import mask

# ===============================
# PATHS
# ===============================

input_tif = "/content/MIVD_POSTMONSOON_SD_1951_2025.tif"
shp_path = "/content/1a_GRB_WGS84.shp"

output_dir = "/content/MVIMD_SINGLE_SD_CLIPPED"
os.makedirs(output_dir, exist_ok=True)

output_tif = os.path.join(
    output_dir,
    "MVIMD_POSTMONSOON_SD.tif"
)

# ===============================
# READ SHAPEFILE
# ===============================

gdf = gpd.read_file(shp_path)
shapes = gdf.geometry.values


# ===============================
# CLIP
# ===============================

with rasterio.open(input_tif) as src:

    clipped, transform = mask(
        src,
        shapes,
        crop=True,
        nodata=src.nodata,
        filled=True
    )

    meta = src.meta.copy()

    meta.update({
        "height": clipped.shape[1],
        "width": clipped.shape[2],
        "transform": transform,
        "nodata": src.nodata
    })

    # Save
    with rasterio.open(output_tif, "w", **meta) as dst:
        dst.write(clipped)


print("✅ FILE CLIPPED SUCCESSFULLY!")
print("📂 OUTPUT:", output_tif)


✅ FILE CLIPPED SUCCESSFULLY!
📂 OUTPUT: /content/MVIMD_SINGLE_SD_CLIPPED/MVIMD_POSTMONSOON_SD.tif


In [15]:
from google.colab import files
files.download("/content/MVIMD_SINGLE_SD_CLIPPED/MVIMD_POSTMONSOON_SD.tif")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [16]:
import os
import geopandas as gpd
import rasterio
from rasterio.mask import mask

# ===============================
# PATHS
# ===============================

shp_path = "/content/1a_GRB_WGS84.shp"

input_files = [
    "/content/PRECIP_POST_SD.tif",
    "/content/PRECIP_MON_SD.tif"
]

output_dir = "/content/PRECIP_SD_CLIPPED"
os.makedirs(output_dir, exist_ok=True)


# ===============================
# READ SHAPEFILE
# ===============================

gdf = gpd.read_file(shp_path)
shapes = gdf.geometry.values


# ===============================
# CLIP FUNCTION
# ===============================

def clip_file(input_tif, output_tif):

    with rasterio.open(input_tif) as src:

        clipped, transform = mask(
            src,
            shapes,
            crop=True,
            nodata=src.nodata,
            filled=True
        )

        meta = src.meta.copy()

        meta.update({
            "height": clipped.shape[1],
            "width": clipped.shape[2],
            "transform": transform,
            "nodata": src.nodata
        })

        with rasterio.open(output_tif, "w", **meta) as dst:
            dst.write(clipped)


# ===============================
# PROCESS FILES
# ===============================

print("🚀 CLIPPING PRECIP SD FILES...\n")

for tif in input_files:

    name = os.path.basename(tif).replace(".tif", "")

    # Rename properly
    if "POST" in name:
        out_name = "PRECIP_POSTMONSOON_SD.tif"
    elif "MON" in name:
        out_name = "PRECIP_MONSOON_SD.tif"
    else:
        out_name = name + "_CLIPPED.tif"

    out_path = os.path.join(output_dir, out_name)

    clip_file(tif, out_path)

    print("✅ DONE:", out_name)


print("\n🎉 ALL PRECIP FILES CLIPPED!")
print("📂 OUTPUT FOLDER:", output_dir)


🚀 CLIPPING PRECIP SD FILES...

✅ DONE: PRECIP_POSTMONSOON_SD.tif
✅ DONE: PRECIP_MONSOON_SD.tif

🎉 ALL PRECIP FILES CLIPPED!
📂 OUTPUT FOLDER: /content/PRECIP_SD_CLIPPED


In [18]:
import os
import geopandas as gpd
import rasterio
from rasterio.mask import mask
import numpy as np

# ===============================
# PATHS
# ===============================

shp_path = "/content/1a_GRB_WGS84.shp"

input_files = [
    "/content/PRECIP_POST_SD.tif",
    "/content/PRECIP_MON_SD.tif"
]

output_dir = "/content/PRECIP_SD_CLIPPED_NO_BG"
os.makedirs(output_dir, exist_ok=True)


# ===============================
# READ SHAPEFILE
# ===============================

gdf = gpd.read_file(shp_path)
shapes = gdf.geometry.values


# ===============================
# CLIP FUNCTION
# ===============================

def clip_file(input_tif, output_tif):

    with rasterio.open(input_tif) as src:

        # Force nodata value
        nodata_val = -9999

        clipped, transform = mask(
            src,
            shapes,
            crop=True,
            nodata=nodata_val,
            filled=True
        )

        meta = src.meta.copy()

        meta.update({
            "height": clipped.shape[1],
            "width": clipped.shape[2],
            "transform": transform,
            "nodata": nodata_val,
            "dtype": "float32"
        })

        # Convert background to nodata
        clipped = clipped.astype("float32")
        clipped[clipped == 0] = nodata_val


        with rasterio.open(output_tif, "w", **meta) as dst:
            dst.write(clipped)


# ===============================
# PROCESS FILES
# ===============================

print("🚀 CLIPPING PRECIP SD FILES (NO BACKGROUND)...\n")

for tif in input_files:

    name = os.path.basename(tif).replace(".tif", "")

    if "POST" in name:
        out_name = "PRECIP_POSTMONSOON_SD.tif"
    elif "MON" in name:
        out_name = "PRECIP_MONSOON_SD.tif"
    else:
        out_name = name + "_CLIPPED.tif"

    out_path = os.path.join(output_dir, out_name)

    clip_file(tif, out_path)

    print("✅ DONE:", out_name)


print("\n🎉 ALL FILES CLIPPED — NO BACKGROUND!")
print("📂 OUTPUT FOLDER:", output_dir)


🚀 CLIPPING PRECIP SD FILES (NO BACKGROUND)...

✅ DONE: PRECIP_POSTMONSOON_SD.tif
✅ DONE: PRECIP_MONSOON_SD.tif

🎉 ALL FILES CLIPPED — NO BACKGROUND!
📂 OUTPUT FOLDER: /content/PRECIP_SD_CLIPPED_NO_BG


In [19]:
from google.colab import files

files.download("/content/PRECIP_SD_CLIPPED_NO_BG/PRECIP_POSTMONSOON_SD.tif")
files.download("/content/PRECIP_SD_CLIPPED_NO_BG/PRECIP_MONSOON_SD.tif")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [20]:
import zipfile
import os

zip_path = "/content/PRECIP_SD_CLIPPED_NO_BG.zip"
folder = "/content/PRECIP_SD_CLIPPED_NO_BG"

with zipfile.ZipFile(zip_path, "w", zipfile.ZIP_DEFLATED) as z:
    for f in os.listdir(folder):
        z.write(os.path.join(folder, f), f)

from google.colab import files
files.download(zip_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [17]:
from google.colab import files

files.download("/content/PRECIP_SD_CLIPPED/PRECIP_POSTMONSOON_SD.tif")
files.download("/content/PRECIP_SD_CLIPPED/PRECIP_MONSOON_SD.tif")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>